In [5]:
import pandas as pd
import numpy as np

### Lecture des fichiers

In [17]:
data_urls = {
    "Élus arrondissements" : "https://www.data.gouv.fr/fr/datasets/r/3b6b2281-b9d9-4959-ae9d-c2c166dff118",
    "Élus municipaux" : "https://www.data.gouv.fr/fr/datasets/r/d5f400de-ae3f-4966-8cb6-a85c70c6c24a",
    "Élus communautaires" : "https://www.data.gouv.fr/fr/datasets/r/41d95d7d-b172-4636-ac44-32656367cdc7",
    "Élus départementaux" : "https://www.data.gouv.fr/fr/datasets/r/601ef073-d986-4582-8e1a-ed14dc857fba",
    "Élus régionaux" : "https://www.data.gouv.fr/fr/datasets/r/430e13f9-834b-4411-a1a8-da0b4b6e715c"
}

dfs = {}

for data_name,url in data_urls.items():
    dfs[data_name] = pd.read_csv(url, sep=";", dtype="str")

> Commentaires : Pour échapper cette erreur :

> C:\Users\lawan\AppData\Local\Temp\ipykernel_6288\1067538451.py:12: DtypeWarning: Columns (0,3,4) have mixed types. Specify dtype option on import or set low_memory=False. dfs[data_name] = pd.read_csv(url, sep=";")
> C:\Users\lawan\AppData\Local\Temp\ipykernel_6288\1067538451.py:12: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False. dfs[data_name] = pd.read_csv(url, sep=";")

> J'ai rajouté dtype = "str". Ainsi, au chargement, tous les colonnes sera considéré comme des strings. Pendant le netttoyage des données, je mettrai les types adéquats.



In [ ]:
for name, data in dfs.items():
    print(name,"\n\n",data.head(5))

Élus arrondissements 
   Code du département Libellé du département Code de la commune  \
0                  13       Bouches-Du-Rhône              13055   
1                  13       Bouches-Du-Rhône              13055   
2                  13       Bouches-Du-Rhône              13055   
3                  13       Bouches-Du-Rhône              13055   
4                  13       Bouches-Du-Rhône              13055   

  Libellé de la commune   Libellé du secteur Nom de l'élu Prénom de l'élu  \
0             Marseille  Marseille Secteur 1        AMICO         Patrick   
1             Marseille  Marseille Secteur 1      ASPINAS          Gilles   
2             Marseille  Marseille Secteur 1       BARLES       Sébastien   
3             Marseille  Marseille Secteur 1       BECARD           Leïla   
4             Marseille  Marseille Secteur 1   BERNASCONI          Sabine   

  Code sexe Date de naissance Lieu de naissance  \
0         M        25/08/1955         MARSEILLE   
1        

### Identification des colonnes en commun

In [73]:
colonnes_liste = {}
for name, df in dfs.items():
    for column in df.columns:
        # Récupérer la valeur de column si elle est déjà dans colonnes_liste, ou lui assigner la valeur {"nom_colonne": column, "occurence_colonne": 1, "fichiers": [name]} par défaut
        colonne_info = colonnes_liste.setdefault(column, {"nom_colonne": column, "occurence_colonne": 1, "fichiers": [name]})
        
        # Rajouter le fichier si il n'est pas encore compté, si non, rien
        if name not in colonne_info["fichiers"]:
            colonne_info["occurence_colonne"] += 1
            colonne_info["fichiers"].append(name)

In [74]:
for keys,value in colonnes_liste.items():
    print(keys, " : ", value)

Code du département  :  {'nom_colonne': 'Code du département', 'occurence_colonne': 4, 'fichiers': ['Élus arrondissements', 'Élus municipaux', 'Élus communautaires', 'Élus départementaux']}
Libellé du département  :  {'nom_colonne': 'Libellé du département', 'occurence_colonne': 4, 'fichiers': ['Élus arrondissements', 'Élus municipaux', 'Élus communautaires', 'Élus départementaux']}
Code de la commune  :  {'nom_colonne': 'Code de la commune', 'occurence_colonne': 2, 'fichiers': ['Élus arrondissements', 'Élus municipaux']}
Libellé de la commune  :  {'nom_colonne': 'Libellé de la commune', 'occurence_colonne': 2, 'fichiers': ['Élus arrondissements', 'Élus municipaux']}
Libellé du secteur  :  {'nom_colonne': 'Libellé du secteur', 'occurence_colonne': 1, 'fichiers': ['Élus arrondissements']}
Nom de l'élu  :  {'nom_colonne': "Nom de l'élu", 'occurence_colonne': 5, 'fichiers': ['Élus arrondissements', 'Élus municipaux', 'Élus communautaires', 'Élus départementaux', 'Élus régionaux']}
Prénom 

In [79]:
# Transformation en dataFrame pandas
colonnes_final = []

for value in colonnes_liste.values():
    new_value = value.copy() # Afin de modifier sans affecter l'original
    new_value["fichiers"] = ",".join(new_value["fichiers"])
    colonnes_final.append(new_value)

colonnes_final = pd.DataFrame(colonnes_final)

print(colonnes_final.head(5))

              nom_colonne  occurence_colonne  \
0     Code du département                  4   
1  Libellé du département                  4   
2      Code de la commune                  2   
3   Libellé de la commune                  2   
4      Libellé du secteur                  1   

                                            fichiers  
0  Élus arrondissements,Élus municipaux,Élus comm...  
1  Élus arrondissements,Élus municipaux,Élus comm...  
2               Élus arrondissements,Élus municipaux  
3               Élus arrondissements,Élus municipaux  
4                               Élus arrondissements  


In [82]:
# Colonnes communes
colonnes_final[colonnes_final["occurence_colonne"]==5]["nom_colonne"]

5                                      Nom de l'élu
6                                   Prénom de l'élu
7                                         Code sexe
8                                 Date de naissance
10       Code de la catégorie socio-professionnelle
11    Libellé de la catégorie socio-professionnelle
12                          Date de début du mandat
13                           Libellé de la fonction
14                     Date de début de la fonction
Name: nom_colonne, dtype: object

> Commentaires : Il semble que les noms des colonnes soit propres. D'une fichier à une autre, il semble ne pas avoir des colonnes, a-priori identiques qui porterait des noms différents.

> **Les colonnes communes** au 5 jeu de données sont :  Nom de l'élu, Prénom de l'élu, Code sexe, Date de naissance, Code de la catégorie socio-professionnelle, Libellé de la catégorie socio-professionnelle, Date de début du mandat, Libellé de la fonction, Date de début de la fonction.

### Fusion des fichiers

In [106]:
dfs_concat = pd.concat(dfs,axis=0)
dfs_concat = dfs_concat.reset_index().rename(columns={"level_0":"Type d'élu"})
dfs_concat = dfs_concat.drop(columns=["level_1"])
print(dfs_concat.head())

             Type d'élu Code du département Libellé du département  \
0  Élus arrondissements                  13       Bouches-Du-Rhône   
1  Élus arrondissements                  13       Bouches-Du-Rhône   
2  Élus arrondissements                  13       Bouches-Du-Rhône   
3  Élus arrondissements                  13       Bouches-Du-Rhône   
4  Élus arrondissements                  13       Bouches-Du-Rhône   

  Code de la commune Libellé de la commune   Libellé du secteur Nom de l'élu  \
0              13055             Marseille  Marseille Secteur 1        AMICO   
1              13055             Marseille  Marseille Secteur 1      ASPINAS   
2              13055             Marseille  Marseille Secteur 1       BARLES   
3              13055             Marseille  Marseille Secteur 1       BECARD   
4              13055             Marseille  Marseille Secteur 1   BERNASCONI   

  Prénom de l'élu Code sexe Date de naissance  ... N° SIREN Libellé de l'EPCI  \
0         Patrick

### Evaluation du jeu de données

In [107]:
dfs_concat.dtypes

Type d'élu                                         object
Code du département                                object
Libellé du département                             object
Code de la commune                                 object
Libellé de la commune                              object
Libellé du secteur                                 object
Nom de l'élu                                       object
Prénom de l'élu                                    object
Code sexe                                          object
Date de naissance                                  object
Lieu de naissance                                  object
Code de la catégorie socio-professionnelle         object
Libellé de la catégorie socio-professionnelle      object
Date de début du mandat                            object
Libellé de la fonction                             object
Date de début de la fonction                       object
Code de la nuance politique                        object
Libellé de la 

In [118]:
lignes = 0
for df in dfs.values():
    lignes += df.shape[0]
    print(df.shape)
print("Nombre de lignes des jeux de données : ", lignes)

(1025, 17)
(489972, 16)
(64885, 17)
(4050, 13)
(1748, 13)
Nombre de lignes des jeux de données :  561680


In [119]:
print("Nombre de lignes des jeux de données après fusion : ",dfs_concat.shape)

Nombre de lignes des jeux de données après fusion :  (561680, 31)


In [ ]:
## Nombre de NA par colonnes

dfs_concat.isna().sum(axis=0)

Type d'élu                                              0
Code du département                                  5366
Libellé du département                               5366
Code de la commune                                  70683
Libellé de la commune                               70683
Libellé du secteur                                 560655
Nom de l'élu                                            1
Prénom de l'élu                                         0
Code sexe                                               0
Date de naissance                                       0
Lieu de naissance                                  560666
Code de la catégorie socio-professionnelle            164
Libellé de la catégorie socio-professionnelle           0
Date de début du mandat                                 0
Libellé de la fonction                             396148
Date de début de la fonction                       396148
Code de la nuance politique                        560655
Libellé de la 

In [128]:
## Nombre de lignes contenant des NA
nb_lignes_NA = dfs_concat.isna().any(axis=1).sum()

print("Nombre de lignes contenant des NA : ", nb_lignes_NA)

Nombre de lignes contenant des NA :  561680


> **Commentaires** : Toutes les lignes contient des NA